In [1]:
# Alright lets see what this is all about
import numpy as np
import pandas as pd
import sklearn 

In [2]:
#LOAD GIS WATERMAIN AND BREAK DATA
df = pd.read_csv('Kirk_All.csv')
#df.dropna(inplace=True) # Are there any columns where we can deal with NA values?
# Can we use imputation instead?
breaks = pd.read_csv('Kirk_Break_Data.csv')
df['INSTYEAR'] = df['INSTYEAR'].map(lambda x: int((str(x)[-12:-8])), na_action='ignore')
#df = pd.get_dummies(df)
df.head()

,FID,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLP_CLASS,FULL_STRNA,SPEED,FUNC_CLASS
0,0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,2.0,120TH AVE NE,25,MINOR ARTERIAL
1,1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,1.0,122ND LN NE,25,NEIGHBORHOOD ACCESS
2,2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,0.0,124TH AVE NE,35,PRINCIPAL ARTERIAL
3,3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,0.0,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL
4,4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,6.0,,,


In [3]:
# EXTRACTING FEATURES
features = ['FID_WA_Mai', 'TYPE', 'LENGTH', 'DIAMETER', 'MATERIAL', 'INSTYEAR', 'MUSYM', 'Dist_Rail', 'SLP_CLASS', 'FULL_STRNA', 'SPEED', 'FUNC_CLASS']
pipes = df[features]
pipes.head()

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLP_CLASS,FULL_STRNA,SPEED,FUNC_CLASS
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,2.0,120TH AVE NE,25,MINOR ARTERIAL
1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,1.0,122ND LN NE,25,NEIGHBORHOOD ACCESS
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,0.0,124TH AVE NE,35,PRINCIPAL ARTERIAL
3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,0.0,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL
4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,6.0,,,


In [4]:

model_feats = [
    'DIAMETER',
    'MATERIAL',
    'INSTYEAR',
    'Dist_Rail',
    'Break_Year'

]
filt_breaks = breaks[model_feats]

filt_breaks.head()


,DIAMETER,MATERIAL,INSTYEAR,Dist_Rail,Break_Year
0,6,CI,NaN,NaN,2004
1,8,AC,NaN,15000.0,2004
2,6,AC,NaN,15000.0,2004
3,8,AC,1976,7000.0,2004
4,12,AC,1968,12000.0,2004


In [5]:
filt_breaks['Break_Year'].map(lambda x: int(x), na_action='ignore')
#filt_breaks.dropna()
filt_breaks = filt_breaks[filt_breaks['INSTYEAR'] != '1959, 1963']
filt_breaks['INSTYEAR'] = filt_breaks['INSTYEAR'].map(lambda x: float(x), na_action='ignore')
filt_breaks['AGE'] = filt_breaks['Break_Year'] - filt_breaks['INSTYEAR']
filt_breaks = filt_breaks.dropna()

In [6]:
# GROUP PIPES BY THEIR UNIQUE ID - NEEDED BECAUSE OF HOW SLOPE BREAKS UP PIPES IN DIFFERENT SLOPE CLASSES
def group_by_id(df):

    grouping = df.groupby('FID_WA_Mai').groups
    unique_pipes = df['FID_WA_Mai'].unique()
    indices = []
    for i in range(len(unique_pipes)):
        pipe = unique_pipes[i]
        val = grouping[pipe][0]
        indices.append(val)
    
    # CALCULATE AVERAGE SLOPES FOR PIPES THAT GOT SPLIT INTO MULTIPLE ROWS
    slopes = df[['FID_WA_Mai', 'SLP_CLASS']]
    slopes = slopes.groupby('FID_WA_Mai')['SLP_CLASS'].mean()
    slopes = pd.Series(slopes)

    filtered = df.loc[indices, 'FID_WA_Mai':'FUNC_CLASS']
    filtered = filtered.drop('SLP_CLASS', axis=1)
    filtered['SLOPE'] = slopes.values

    return filtered

In [7]:
filtered = group_by_id(pipes)
filtered.head()

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,120TH AVE NE,25,MINOR ARTERIAL,2.000000
1,2,FIRE,2.0,2.0,UNK,1992.0,AgC,7000,122ND LN NE,25,NEIGHBORHOOD ACCESS,1.000000
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,124TH AVE NE,35,PRINCIPAL ARTERIAL,0.000000
3,4,FIRE,3.0,6.0,DI,2005.0,Ur,4000,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL,0.000000
4,5,FIRE,149.0,8.0,AC,1963.0,Ur,7000,,,,3.888889


In [8]:
more_feats = [
    'DIAMETER',
    'MATERIAL',
    'INSTYEAR',
    'MUSYM',
    'Dist_Rail',
]

filtered_pipes = filtered[more_feats]
filtered_pipes.head()

,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail
0,8.0,UNK,NaN,Ur,6000
1,2.0,UNK,1992.0,AgC,7000
2,0.0,UNK,NaN,AgC,6000
3,6.0,DI,2005.0,Ur,4000
4,8.0,AC,1963.0,Ur,7000


In [204]:
break_1_9_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W')] #& (filtered['INSTYEAR'] < 2005) & (filtered['INSTYEAR'] > 2003)]
break_1_9_04 = break_1_9_04[break_1_9_04['INSTYEAR'] == 2004]
break_1_9_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004) & (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 8.0)] 
nearby

similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [205]:
break_1_21_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] == 2004)]
break_1_21_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004)] #& (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 6.0)] 


similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [197]:
break_2_25_04 = filtered[(filtered['FULL_STRNA'] == 'NE 59TH ST')]
break_2_25_04

# THIS WILL LIKELY NEED TO BE altered in data set

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
22011,3634,DIST,1.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
22012,3635,DIST,5.0,12.0,AC,1971,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
39310,6927,DIST,261.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,2.586207


In [42]:
break_1_04_07 = pipes[(pipes['INSTYEAR'] == 2007) & (pipes['FULL_STRNA'].str.contains('2ND'))]
(break_1_04_07)

nearby = pipes[(pipes['INSTYEAR'] < 2007) & (pipes['DIAMETER'] == 4) & (pipes['MATERIAL'] == 'AC') & (pipes['FULL_STRNA'].str.contains('2ND AVE')) & (pipes['Dist_Rail'] == 12000)]
nearby

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
26578,4438,DIST,197.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,1.5
40951,7457,DIST,164.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,3.0
40953,7458,DIST,2.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,0.0
40954,7459,DIST,3.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,1.5
40955,7460,DIST,2.0,4.0,AC,1991.0,AmC,12000,2ND AVE S,25,NEIGHBORHOOD ACCESS,0.5


In [52]:
break_10_08_07 = pipes[(pipes['INSTYEAR'] == 2009) & (pipes['FULL_STRNA'].str.contains('9TH AVE'))]
break_10_08_07

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
15914,2593,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
15915,2594,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
16042,2606,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
23601,3893,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
24474,4073,DIST,4.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,0.8
24507,4078,DIST,59.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
24510,4079,DIST,415.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0
24527,4081,DIST,9.0,12.0,DI,2009.0,AmC,12000,9TH AVE,25,NEIGHBORHOOD ACCESS,1.0


In [69]:
break_11_2_07 = pipes[(pipes['INSTYEAR'] == 2008) & (pipes['FULL_STRNA'] == '20TH AVE')]
break_11_2_07

nearby = pipes[(pipes['INSTYEAR'] < 2007) & (pipes['FULL_STRNA'] == '20TH AVE')]
nearby

similar = pipes[(pipes['MATERIAL'] == 'AC') & (pipes['DIAMETER'] == 4) & (pipes['MUSYM'] == 'InC') & (pipes['Dist_Rail'] >= 10000)]
nearby

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
9492,1616,DIST,622.0,6.0,AC,1968.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,1.058824
9509,1617,DIST,1.0,6.0,AC,1968.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,0.000000
10655,1820,DIST,182.0,8.0,DI,1993.0,InC,11000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.125000
10663,1821,DIST,272.0,8.0,DI,1993.0,InC,11000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.800000
10698,1823,DIST,32.0,6.0,AC,1968.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,0.000000
15501,2536,DIST,270.0,8.0,DI,2005.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.600000
35749,6006,DIST,12.0,8.0,DI,2005.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,3.000000
35750,6007,DIST,294.0,8.0,DI,1993.0,InC,12000,20TH AVE,25,NEIGHBORHOOD ACCESS,2.000000
36467,6138,DIST,592.0,8.0,DI,1997.0,AmC,13000,20TH AVE,25,NEIGHBORHOOD ACCESS,1.842105
